In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
sys.path.insert(1, '/home/brodt/.local/lib/python3.6/site-packages')

In [2]:
import torch
import cv2
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

from submission_proto import motion_submission_pb2
from train import N_TRAJS, TL, get_model, RESIZE

In [3]:
class WaymoLoader(Dataset):
    def __init__(self, directory):
        files = os.listdir(directory)
        self.files = [directory + f for f in files if f.split(".")[-1] == "npz"]
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
            filename = self.files[idx]
            data = np.load(filename, allow_pickle=True)

            raster = data["raster"]
            raster = torch.tensor(raster.transpose(2,1,0)) / 255.
#             raster = RESIZE(raster)

            center = data["shift"]
            yaw = data["yaw"]
            agent_id = data["object_id"]
            scenario_id = data["scenario_id"]

            return raster, center, yaw, agent_id, str(scenario_id), data["_gt_marginal"], data["gt_marginal"]

In [6]:
model = get_model("xception71", in_ch=47).cuda()  # seresnext50_32x4d
checkpoint = torch.load("./test_xception71_263000_dev_131.pth")
# model = get_model("xception71", in_ch=25).cuda()  # seresnext50_32x4d
# checkpoint = torch.load("/home/brodt/kaggle/waymo/xception71_327000_val2_144.127.pth")

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

XceptionAligned(
  (stem): Sequential(
    (0): ConvBnAct(
      (conv): Conv2d(47, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (act): ReLU(inplace=True)
      )
    )
    (1): ConvBnAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (act): ReLU(inplace=True)
      )
    )
  )
  (blocks): Sequential(
    (0): XceptionModule(
      (shortcut): ConvBnAct(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNormAct2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stack): Sequential(
        (act1): ReLU()
        (conv1): SeparableConv2d(
          (conv_dw): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [5]:
dataset = WaymoLoader("/home/brodt/kaggle/waymo/test/")
loader = DataLoader(dataset, batch_size=128, num_workers=12)

In [6]:
# RES_dev

In [7]:
RES = {}
selector = np.arange(1, 81)[4::5] - 1

In [8]:
with torch.no_grad():
    for raster, center, yaw, agent_id, scenario_id, gt_abs, gt_loc in tqdm(loader):
        outputs = model(raster.cuda())
        confidences, pred = outputs[:,:N_TRAJS], outputs[:,N_TRAJS:]
        confidences = torch.softmax(confidences, dim=1)
        bs = raster.shape[0]
        pred = pred.view(bs, N_TRAJS, TL, 2)
        
        pred = pred.cpu().numpy()
        confidences = confidences.cpu().numpy()
        agent_id = agent_id.cpu().numpy()
        center = center.cpu().numpy()
        yaw = yaw.cpu().numpy()
        
        for p, conf, aid, sid, c, y in zip(pred, confidences, agent_id, scenario_id, center, yaw):
            if sid not in RES:
                RES[sid] = []
#                 rot_matrix = np.array([
#                     [np.cos(-y), -np.sin(-y)],
#                     [np.sin(-y),  np.cos(-y)],
#                 ]).reshape(2, 2)

            RES[sid].append(
                {
                    "aid": aid,
                    "conf": conf,
                    "pred": p,  # [i][selector] @ rot_matrix + c,
                    "yaw": -y,
                    "center": c
                }
            )

100%|██████████| 1532/1532 [34:10<00:00,  1.34s/it]


In [13]:
motion_challenge_submission = motion_submission_pb2.MotionChallengeSubmission()
motion_challenge_submission.account_name = "cyrill.brodt@gmail.com"
motion_challenge_submission.authors.extend([
    "konevman@gmail.com", "enorone@gmail.com", "cyrill.brodt@gmail.com",
])
motion_challenge_submission.submission_type = motion_submission_pb2.MotionChallengeSubmission.SubmissionType.MOTION_PREDICTION
motion_challenge_submission.unique_method_name = "SimpleCNNOnRaster"

for scenario_id, data in tqdm(RES.items()):
    scenario_predictions = motion_challenge_submission.scenario_predictions.add()
    scenario_predictions.scenario_id = scenario_id
    prediction_set = scenario_predictions.single_predictions
    
    for jind, d in enumerate(data):
        predictions = prediction_set.predictions.add()
        predictions.object_id = int(d["aid"])
        
        y = d["yaw"]
        rot_matrix = np.array([
            [np.cos(y), -np.sin(y)],
            [np.sin(y),  np.cos(y)],
        ]).reshape(2, 2)
        
#         for i in range(N_TRAJS):
        for i in np.argsort(-d["conf"]):
            scored_trajectory = predictions.trajectories.add()
            scored_trajectory.confidence = d["conf"][i]
            
            trajectory = scored_trajectory.trajectory
            
            p = d["pred"][i][selector] @ rot_matrix + d["center"]
            
            trajectory.center_x.extend(p[:, 0])
            trajectory.center_y.extend(p[:, 1])

100%|██████████| 44926/44926 [00:44<00:00, 1020.97it/s]


In [14]:
sub_name = "submission_test_xception71_263000_dev_131.729.proto"
f = open(sub_name, "wb")
f.write(motion_challenge_submission.SerializeToString())
f.close()

In [15]:
!tar -czf submission_test_xception71_263000_dev_131.729.tar.gz submission_test_xception71_263000_dev_131.729.proto

In [ ]:
1